# Proyecto EAF - Notebook Monolítico

## Electric Arc Furnace - Predicción de Temperatura y Composición Química

### Configuracion

In [ ]:
!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost joblib kagglehub

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path
import json
from typing import Dict, List, Tuple, Optional, Any
import logging
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import kagglehub
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBRegressor


In [ ]:
# Configurar logging para que imprima en la salida del notebook con formato de tiempo
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True  # Forzar reconfiguración si ya existe
)

# Crear logger principal del notebook
logger = logging.getLogger('EAF_Notebook')
logger.setLevel(logging.INFO)

#### 1.2 Configuración de Directorios

Definimos la estructura de carpetas del proyecto y las creamos automáticamente si no existen:
- `data/raw`: Datos crudos descargados de Kaggle
- `data/processed`: Datos procesados listos para entrenamiento
- `models`: Modelos de temperatura entrenados
- `models/chemical_results`: Modelos químicos y sus métricas

In [ ]:
# =============================================================================
# CONFIGURACIÓN DE DIRECTORIOS
# =============================================================================

# Definir raíz del proyecto usando pathlib
# Usamos Path.cwd() para notebooks, asumiendo que se ejecuta desde la raíz del proyecto
PROJECT_ROOT = Path.cwd()

# Si el notebook está en una subcarpeta, ajustar:
# PROJECT_ROOT = Path.cwd().parent  # Descomentar si es necesario

# Definir estructura de directorios
DIRECTORIES = {
    'DATA_RAW': PROJECT_ROOT / 'data' / 'raw',
    'DATA_PROCESSED': PROJECT_ROOT / 'data' / 'processed',
    'MODELS': PROJECT_ROOT / 'models',
    'CHEMICAL_RESULTS': PROJECT_ROOT / 'models' / 'chemical_results'
}

# Crear directorios si no existen
for dir_name, dir_path in DIRECTORIES.items():
    dir_path.mkdir(parents=True, exist_ok=True)
    logger.info(f"Directorio verificado/creado: {dir_name} -> {dir_path}")

# Asignar a variables individuales para fácil acceso
DATA_RAW = DIRECTORIES['DATA_RAW']
DATA_PROCESSED = DIRECTORIES['DATA_PROCESSED']
MODELS_DIR = DIRECTORIES['MODELS']
CHEMICAL_RESULTS_DIR = DIRECTORIES['CHEMICAL_RESULTS']

print(f"\n{'='*60}")
print("ESTRUCTURA DE DIRECTORIOS DEL PROYECTO")
print(f"{'='*60}")
print(f"PROJECT_ROOT:      {PROJECT_ROOT}")
print(f"DATA_RAW:          {DATA_RAW}")
print(f"DATA_PROCESSED:    {DATA_PROCESSED}")
print(f"MODELS_DIR:        {MODELS_DIR}")
print(f"CHEMICAL_RESULTS:  {CHEMICAL_RESULTS_DIR}")
print(f"{'='*60}")

---

## PARTE 2: Ingesta de Datos

En esta sección implementamos la descarga automática del dataset desde Kaggle.

El dataset **"Industrial Data from the Arc Furnace"** contiene 11 archivos CSV con información del proceso de fundición:
- Datos del transformador y temperatura
- Mediciones químicas iniciales y finales
- Materiales cargados e inyectados
- Datos del horno de cuchara (Ladle Furnace)

### 2.1 Configuración de Descarga y Archivos Esperados

Definimos los archivos que componen el dataset completo y la referencia al dataset de Kaggle.

In [ ]:
KAGGLE_DATASET = "yuriykatser/industrial-data-from-the-arc-furnace"

# ARCHIVOS ESPERADOS DEL DATASET
ARCHIVOS_ESPERADOS = [
    "eaf_transformer.csv",              # Datos del transformador del horno
    "basket_charged.csv",               # Cestas de chatarra cargadas
    "eaf_temp.csv",                      # Mediciones de temperatura
    "eaf_final_chemical_measurements.csv",  # Composición química final
    "eaf_added_materials.csv",           # Materiales añadidos al horno
    "inj_mat.csv",                       # Materiales inyectados
    "eaf_gaslance_mat.csv",              # Gases inyectados por lanza
    "lf_initial_chemical_measurements.csv",  # Química inicial (horno cuchara)
    "ladle_tapping.csv",                 # Datos de colada
    "lf_added_materials.csv",            # Materiales añadidos en LF
    "ferro.csv"                          # Ferroaleaciones utilizadas
]

### 2.2 Función de Descarga de Datos

Implementamos `download_data()` con la siguiente lógica:
1. Verifica si todos los archivos ya existen en `data/raw`
2. Si existen y `force=False`, no descarga (evita trabajo innecesario)
3. Si faltan archivos, descarga desde Kaggle usando `kagglehub`
4. Copia los archivos al directorio del proyecto
5. Reporta el estado de cada archivo con su tamaño

In [ ]:

faltan_datos = any(not (DATA_RAW / f).exists() for f in ARCHIVOS_ESPERADOS)

print(f"⬇️ Descargando {KAGGLE_DATASET}...")
try:
    # Descarga a caché de Kaggle
    cached_path = Path(kagglehub.dataset_download(KAGGLE_DATASET))
    DATA_RAW.mkdir(parents=True, exist_ok=True)
    
    # Mover archivos a nuestra carpeta raw
    for archivo in ARCHIVOS_ESPERADOS:
        shutil.copy2(cached_path / archivo, DATA_RAW / archivo)
    print("✅ Descarga y copia completada.")
    
except Exception as e:
    print(f"❌ Error durante la descarga: {e}")
    raise

print(f"📂 Ruta de datos: {DATA_RAW}")

### Funciones Auxiliares de Preprocesamiento

Las siguientes funciones se utilizan tanto para el dataset secuencial como para extraer variables estáticas por colada:

- `load_standardized()`: Carga CSVs con conversión automática de formato europeo
- `aggregate_*()`: Funciones de agregación de series temporales
- `pivot_materials()`: Pivotado de materiales añadidos
- `build_master_dataset()`: Construcción del dataset maestro de variables estáticas

Estas funciones son **reutilizadas** por la PARTE 3 para fusionar variables estáticas con el dataset secuencial.

#### Función de Carga Estandarizada

Esta función es fundamental: carga CSVs y convierte automáticamente formatos numéricos europeos (coma decimal) a formato estándar (punto decimal).

In [ ]:
# =============================================================================
# FUNCIÓN DE CARGA ESTANDARIZADA
# =============================================================================

def load_standardized(filepath: Path) -> pd.DataFrame:
    """
    Carga un CSV y estandariza los nombres de columnas.
    
    IMPORTANTE: Detecta y convierte automáticamente formatos numéricos europeos
    donde se usa coma como separador decimal (ej: "12,5" -> 12.5).
    
    Args:
        filepath: Ruta al archivo CSV (Path o string)
    
    Returns:
        DataFrame con:
        - Columnas en minúsculas y sin espacios
        - Valores numéricos con formato decimal estándar (punto)
    
    Example:
        >>> df = load_standardized(DATA_RAW / "eaf_temp.csv")
        >>> df.columns  # ['heatid', 'temp', 'datetime', ...]
    """
    # Cargar CSV
    df = pd.read_csv(filepath, low_memory=False)
    
    # Estandarizar nombres de columnas: minúsculas y sin espacios
    df.columns = df.columns.str.lower().str.strip()
    
    # ---------------------------------------------------------------------
    # CONVERSIÓN DE COMAS DECIMALES (formato europeo -> americano)
    # Detecta columnas tipo object que contienen patrones como "12,5" o "-3,14"
    # ---------------------------------------------------------------------
    for col in df.select_dtypes(include=['object']).columns:
        # Patrón: número opcional negativo, dígitos, coma, dígitos
        # Ejemplos válidos: "12,5", "-3,14", "0,001"
        if df[col].astype(str).str.match(r'^-?\d+,\d+$').any():
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
            logger.debug(f"  Convertida coma decimal en columna: {col}")
    
    return df


# Test de la función
print("✅ Función load_standardized() definida")
print("\nCaracterísticas:")
print("  - Convierte nombres de columnas a minúsculas")
print("  - Elimina espacios en nombres de columnas")
print("  - Detecta y convierte comas decimales europeas a puntos")

#### Funciones de Agregación de Series Temporales

Los datos originales son series temporales (múltiples registros por colada). Estas funciones agregan los datos para obtener **un valor por colada (heatid)**.

In [ ]:
# =============================================================================
# FUNCIONES DE AGREGACIÓN DE SERIES TEMPORALES
# =============================================================================

def aggregate_gas_data(df_gas: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos de gas lance por colada.
    
    Obtiene el ÚLTIMO valor registrado temporalmente (por revtime) de cada colada.
    Esto representa el estado final de O2 y gas inyectados.
    
    Args:
        df_gas: DataFrame con datos de eaf_gaslance_mat.csv
    
    Returns:
        DataFrame indexado por heatid con columnas:
        - total_o2_lance: Último valor de O2 inyectado
        - total_gas_lance: Último valor de gas inyectado
    """
    df = df_gas.copy()
    
    # Convertir columnas a numérico
    cols_gas = ['o2_amount', 'gas_amount']
    for col in cols_gas:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convertir tiempo (formato original: "2016-01-01 18:31:46,003")
    # Nota: La coma en milisegundos debe convertirse a punto
    df['revtime'] = pd.to_datetime(
        df['revtime'].astype(str).str.replace(',', '.', regex=False),
        format='%Y-%m-%d %H:%M:%S.%f',
        errors='coerce'
    )
    
    # Ordenar por tiempo y obtener el ÚLTIMO registro por colada
    df = df.sort_values('revtime')
    grp_gas = df.groupby('heatid').last()[cols_gas].rename(columns={
        'o2_amount': 'total_o2_lance',
        'gas_amount': 'total_gas_lance'
    })
    
    return grp_gas


def aggregate_injection_data(df_inj: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos de inyecciones de carbón por colada.
    
    Obtiene el ÚLTIMO valor registrado temporalmente (por revtime) de cada colada.
    
    Args:
        df_inj: DataFrame con datos de inj_mat.csv
    
    Returns:
        DataFrame indexado por heatid con columna:
        - total_injected_carbon: Último valor de carbón inyectado
    """
    df = df_inj.copy()
    
    # Convertir a numérico
    df['inj_amount_carbon'] = pd.to_numeric(df['inj_amount_carbon'], errors='coerce')
    
    # Convertir tiempo (formato: "2016-01-01 18:31:46,003")
    df['revtime'] = pd.to_datetime(
        df['revtime'].astype(str).str.replace(',', '.', regex=False),
        format='%Y-%m-%d %H:%M:%S.%f',
        errors='coerce'
    )
    
    # Ordenar por tiempo y obtener el ÚLTIMO registro por colada
    df = df.sort_values('revtime')
    grp_inj = df.groupby('heatid').last()[['inj_amount_carbon']].rename(
        columns={'inj_amount_carbon': 'total_injected_carbon'}
    )
    
    return grp_inj


def aggregate_transformer_data(df_transformer: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos del transformador por colada.
    
    Calcula la ENERGÍA TOTAL consumida: MW * Duración (en minutos)
    y la duración total del proceso.
    
    Args:
        df_transformer: DataFrame con datos de eaf_transformer.csv
    
    Returns:
        DataFrame indexado por heatid con columnas:
        - total_energy: Suma de (MW * duración) para toda la colada
        - total_duration: Duración total en minutos
    """
    df = df_transformer.copy()
    
    # Función para parsear DURATION de formato "MM: SS" a minutos decimales
    def parse_duration(duration_str):
        """Convierte 'MM: SS' a minutos decimales."""
        try:
            duration_str = str(duration_str).strip()
            parts = duration_str.split(':')
            if len(parts) == 2:
                minutes = float(parts[0].strip())
                seconds = float(parts[1].strip())
                return minutes + seconds / 60.0
            return 0.0
        except (ValueError, AttributeError):
            return 0.0
    
    # Aplicar conversión de duración
    df['duration_minutes'] = df['duration'].apply(parse_duration)
    
    # Convertir MW a numérico
    df['mw'] = pd.to_numeric(df['mw'], errors='coerce').fillna(0)
    
    # Calcular energía = MW * duración (en minutos)
    df['energy'] = df['mw'] * df['duration_minutes']
    
    # Agregar por colada: SUMA de energía y duración
    grp_transformer = df.groupby('heatid').agg({
        'energy': 'sum',
        'duration_minutes': 'sum'
    }).rename(columns={
        'energy': 'total_energy',
        'duration_minutes': 'total_duration'
    })
    
    return grp_transformer


def aggregate_charged_amount(df_ladle: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega la cantidad total de material cargado por colada.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
    
    Returns:
        DataFrame indexado por heatid con columna:
        - total_charged_amount: Suma total de carga
    """
    df = df_ladle.copy()
    
    # Convertir a numérico
    df['charge_amount'] = pd.to_numeric(df['charge_amount'], errors='coerce').fillna(0)
    
    # Agregar: SUMA por colada
    grp_charged = df.groupby('heatid').agg({
        'charge_amount': 'sum'
    }).rename(columns={'charge_amount': 'total_charged_amount'})
    
    return grp_charged


print("✅ Funciones de agregación definidas:")
print("  - aggregate_gas_data(): O2 y gas inyectado (último valor)")
print("  - aggregate_injection_data(): Carbón inyectado (último valor)")
print("  - aggregate_transformer_data(): Energía total (MW * tiempo)")
print("  - aggregate_charged_amount(): Carga total (suma)")

#### Función de Pivotado de Materiales

Esta función transforma la tabla de materiales añadidos en columnas individuales.
Selecciona los **top N materiales más frecuentes** y crea una columna por cada uno (`added_mat_XXXXXX`).

In [ ]:
# =============================================================================
# FUNCIÓN DE PIVOTADO DE MATERIALES
# =============================================================================

def pivot_materials(df_ladle: pd.DataFrame, top_n: int = 10) -> pd.DataFrame:
    """
    Pivota los materiales agregados, seleccionando los top_n más frecuentes.
    
    Transforma una tabla con múltiples filas por colada (una por material)
    en una tabla con UNA fila por colada y UNA columna por material.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
                  Debe contener columnas: heatid, mat_code, charge_amount
        top_n: Número de materiales más frecuentes a incluir (default: 10)
    
    Returns:
        DataFrame pivotado indexado por heatid con columnas:
        - added_mat_XXXXXX: Cantidad de material con código XXXXXX
        
    Example:
        >>> pivot = pivot_materials(df_ladle, top_n=10)
        >>> pivot.columns
        Index(['added_mat_140107', 'added_mat_202007', ...])
    """
    df = df_ladle.copy()
    
    # Convertir cantidad a numérico
    df['charge_amount'] = pd.to_numeric(df['charge_amount'], errors='coerce')
    
    # Seleccionar los TOP N materiales por frecuencia de uso
    top_materials = df['mat_code'].value_counts().head(top_n).index
    logger.info(f"Top {top_n} materiales seleccionados: {list(top_materials)}")
    
    # Filtrar solo los materiales más frecuentes
    df_filtered = df[df['mat_code'].isin(top_materials)]
    
    # Crear pivot table
    # - index: heatid (una fila por colada)
    # - columns: mat_code (una columna por material)
    # - values: charge_amount (suma de cantidades)
    # - fill_value: 0 (si no se usó el material, es 0)
    pivot_ladle = df_filtered.pivot_table(
        index='heatid',
        columns='mat_code',
        values='charge_amount',
        aggfunc='sum',
        fill_value=0
    ).add_prefix('added_mat_')
    
    logger.info(f"Pivot de materiales: {pivot_ladle.shape[0]} coladas, {pivot_ladle.shape[1]} materiales")
    
    return pivot_ladle


def get_datetime_range(df_ladle: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae el rango de fechas (inicio y fin) de cada colada.
    
    Útil para análisis temporal y filtrado por período.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - fecha_inicio: Primera fecha registrada
        - fecha_fin: Última fecha registrada
    """
    df = df_ladle.copy()
    
    # Convertir a datetime
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    
    # Agregar: mínimo y máximo por colada
    datetime_range = df.groupby('heatid').agg({
        'datetime': ['min', 'max']
    })
    datetime_range.columns = ['fecha_inicio', 'fecha_fin']
    datetime_range = datetime_range.reset_index()
    
    return datetime_range


print("✅ Funciones de pivotado definidas:")
print("  - pivot_materials(): Crea columnas por material (top N)")
print("  - get_datetime_range(): Extrae rango de fechas por colada")

#### Funciones de Extracción de Targets

Estas funciones extraen las variables objetivo (targets) que queremos predecir:
- **Temperatura final**: La última medición de temperatura antes del vaciado
- **Composición química final**: Valores de C, Mn, Si, P, S, Cu, Cr, Mo, Ni

In [ ]:
# =============================================================================
# FUNCIONES DE EXTRACCIÓN DE TARGETS
# =============================================================================

def get_final_temperature(df_temp: pd.DataFrame) -> pd.DataFrame:
    """
    Obtiene la temperatura final (al vaciado) de cada colada.
    
    Toma la ÚLTIMA medición de temperatura registrada temporalmente,
    que corresponde al momento del vaciado del horno.
    
    Args:
        df_temp: DataFrame con datos de eaf_temp.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - target_temperature: Temperatura final en °C
    """
    df = df_temp.copy()
    
    # Detectar columnas automáticamente
    cols_temp = [c for c in df.columns if 'temp' in c and 'time' not in c]
    cols_time = [c for c in df.columns if 'time' in c or 'date' in c]
    
    col_temp_name = cols_temp[0] if cols_temp else 'temp'
    col_time_name = cols_time[0] if cols_time else 'datetime'
    
    logger.info(f"Columna de temperatura detectada: {col_temp_name}")
    logger.info(f"Columna de tiempo detectada: {col_time_name}")
    
    # Limpiar tipos
    df[col_temp_name] = pd.to_numeric(df[col_temp_name], errors='coerce')
    df[col_time_name] = pd.to_datetime(df[col_time_name], errors='coerce')
    
    # Obtener la ÚLTIMA medición (temperatura al vaciado)
    # Ordenar por tiempo y tomar el último registro de cada colada
    df_target = df.sort_values(col_time_name).groupby('heatid').tail(1)
    
    # Seleccionar solo ID y temperatura
    df_target = df_target[['heatid', col_temp_name]].rename(
        columns={col_temp_name: 'target_temperature'}
    )
    
    logger.info(f"Temperaturas extraídas: {len(df_target)} coladas")
    
    return df_target


def get_final_chemical_composition(df_chem_final: pd.DataFrame) -> pd.DataFrame:
    """
    Obtiene la composición química final de cada colada.
    
    Extrae los valores finales de los elementos químicos relevantes
    para el control de calidad del acero.
    
    Args:
        df_chem_final: DataFrame con datos de eaf_final_chemical_measurements.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - target_valc: Carbono final (%)
        - target_valmn: Manganeso final (%)
        - target_valsi: Silicio final (%)
        - target_valp: Fósforo final (%)
        - target_vals: Azufre final (%)
        - target_valcu: Cobre final (%)
        - target_valcr: Cromo final (%)
        - target_valmo: Molibdeno final (%)
        - target_valni: Níquel final (%)
    """
    # Elementos químicos a extraer como targets (lista completa)
    chemical_elements = [
        'valc', 'valmn', 'valsi', 'valp', 'vals',
        'valcu', 'valcr', 'valmo', 'valni'
    ]
    
    # Verificar qué columnas existen realmente en el archivo
    available_elements = [col for col in chemical_elements if col in df_chem_final.columns]
    
    if not available_elements:
        logger.warning("No se encontraron columnas de elementos químicos en el archivo")
        return pd.DataFrame()
    
    logger.info(f"Elementos químicos disponibles: {available_elements}")
    
    # Seleccionar heatid y elementos químicos
    cols_to_select = ['heatid'] + available_elements
    df_targets = df_chem_final[cols_to_select].copy()
    
    # Convertir a numérico (maneja tanto puntos como comas)
    for col in available_elements:
        df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce')
    
    # Renombrar con prefijo target_
    rename_dict = {col: f'target_{col}' for col in available_elements}
    df_targets = df_targets.rename(columns=rename_dict)
    
    # Eliminar duplicados por heatid (tomar el último registro)
    df_targets = df_targets.drop_duplicates(subset=['heatid'], keep='last')
    
    logger.info(f"Targets químicos extraídos: {len(df_targets)} coladas, {len(available_elements)} elementos")
    
    return df_targets


print("✅ Funciones de targets definidas:")
print("  - get_final_temperature(): Última temperatura por colada")
print("  - get_final_chemical_composition(): Composición química final (9 elementos)")

#### Construcción del Dataset Maestro

Estas funciones combinan todas las fuentes de datos en un único dataset maestro, realizando los merges necesarios y la limpieza final.

In [ ]:
# =============================================================================
# CONSTRUCCIÓN DEL DATASET MAESTRO
# =============================================================================

def _filter_outliers_by_target(df: pd.DataFrame, target: str, lower_q: float = 0.01, upper_q: float = 0.99) -> Tuple[pd.DataFrame, int]:
    """
    Filtra outliers en el target especificado usando cuantiles (rows drop).
    Retorna el DataFrame filtrado y el número de filas eliminadas.
    """
    if target not in df.columns or len(df) < 100:
        return df, 0

    try:
        y = df[target].dropna()
        if len(y) < 100:
            return df, 0

        lower_bound = y.quantile(lower_q)
        upper_bound = y.quantile(upper_q)

        # Máscara para conservar valores dentro del rango
        mask = (df[target] >= lower_bound) & (df[target] <= upper_bound)

        # Se mantiene la fila si no es un outlier O si es NaN (será imputado/eliminado después)
        rows_to_keep = df[target].isnull() | mask

        n_removed = (~rows_to_keep).sum()
        df_filtered = df[rows_to_keep]

        return df_filtered, n_removed
    except Exception as e:
        logger.warning(f"Error al filtrar outliers en {target}: {e}")
        return df, 0


def build_master_dataset(raw_data_dir: Path) -> pd.DataFrame:
    """
    Construye el dataset maestro combinando todas las fuentes de datos.

    Este es el NÚCLEO del feature engineering. Carga todos los archivos,
    aplica las agregaciones y fusiona todo en un único DataFrame.

    Args:
        raw_data_dir: Ruta al directorio con los datos raw

    Returns:
        DataFrame maestro con todas las features de input (sin targets)

    Pipeline:
        1. Cargar archivos CSV estandarizados
        2. Agregar series temporales (gas, inyección, transformador, carga)
        3. Pivotar materiales
        4. Extraer rango de fechas
        5. Fusionar todo por heatid
        6. Rellenar nulos técnicos con 0
    """
    logger.info("=" * 50)
    logger.info("CONSTRUYENDO DATASET MAESTRO")
    logger.info("=" * 50)

    # -------------------------------------------------------------------------
    # PASO 1: Cargar archivos necesarios
    # -------------------------------------------------------------------------
    logger.info("Cargando archivos...")

    df_gas = load_standardized(raw_data_dir / "eaf_gaslance_mat.csv")
    logger.info(f"  - eaf_gaslance_mat.csv: {df_gas.shape}")

    df_inj = load_standardized(raw_data_dir / "inj_mat.csv")
    logger.info(f"  - inj_mat.csv: {df_inj.shape}")

    df_ladle = load_standardized(raw_data_dir / "ladle_tapping.csv")
    logger.info(f"  - ladle_tapping.csv: {df_ladle.shape}")

    df_chem_initial = load_standardized(raw_data_dir / "lf_initial_chemical_measurements.csv")
    logger.info(f"  - lf_initial_chemical_measurements.csv: {df_chem_initial.shape}")

    df_transformer = load_standardized(raw_data_dir / "eaf_transformer.csv")
    logger.info(f"  - eaf_transformer.csv: {df_transformer.shape}")

    # -------------------------------------------------------------------------
    # PASO 2: Agregar series temporales
    # -------------------------------------------------------------------------
    logger.info("\nAgregando series temporales...")

    grp_gas = aggregate_gas_data(df_gas)
    logger.info(f"  - Gases: {grp_gas.shape[0]} coladas")

    grp_inj = aggregate_injection_data(df_inj)
    logger.info(f"  - Inyecciones: {grp_inj.shape[0]} coladas")

    grp_transformer = aggregate_transformer_data(df_transformer)
    logger.info(f"  - Transformador: {grp_transformer.shape[0]} coladas")

    grp_charged = aggregate_charged_amount(df_ladle)
    logger.info(f"  - Carga: {grp_charged.shape[0]} coladas")

    # -------------------------------------------------------------------------
    # PASO 3: Pivotar materiales
    # -------------------------------------------------------------------------
    logger.info("\nPivotando materiales...")
    pivot_ladle = pivot_materials(df_ladle, top_n=10)

    # -------------------------------------------------------------------------
    # PASO 4: Extraer rango de fechas
    # -------------------------------------------------------------------------
    logger.info("\nExtrayendo rango de fechas...")
    datetime_range = get_datetime_range(df_ladle)

    # -------------------------------------------------------------------------
    # PASO 5: Fusionar dataset maestro
    # -------------------------------------------------------------------------
    logger.info("\nFusionando dataset maestro...")

    # Dataset base: mediciones químicas iniciales
    df_master = df_chem_initial.copy()
    logger.info(f"  Base (química inicial): {df_master.shape}")

    # Merges (left joins para preservar todos los registros base)
    df_master = df_master.merge(grp_gas, on='heatid', how='left')
    df_master = df_master.merge(grp_inj, on='heatid', how='left')
    df_master = df_master.merge(grp_transformer, on='heatid', how='left')
    df_master = df_master.merge(grp_charged, on='heatid', how='left')
    df_master = df_master.merge(pivot_ladle, on='heatid', how='left')
    df_master = df_master.merge(datetime_range, on='heatid', how='left')

    logger.info(f"  Después de merges: {df_master.shape}")

    # -------------------------------------------------------------------------
    # PASO 6: Rellenar nulos técnicos
    # -------------------------------------------------------------------------
    cols_to_fix = [
        'total_o2_lance', 'total_gas_lance', 'total_injected_carbon',
        'total_energy', 'total_duration', 'total_charged_amount'
    ]
    # Solo rellenar las columnas que existen
    cols_to_fix = [c for c in cols_to_fix if c in df_master.columns]
    df_master[cols_to_fix] = df_master[cols_to_fix].fillna(0)

    # Rellenar columnas de materiales con 0
    mat_cols = [c for c in df_master.columns if c.startswith('added_mat_')]
    df_master[mat_cols] = df_master[mat_cols].fillna(0)

    logger.info(f"\nDataset maestro (inputs): {df_master.shape}")
    logger.info(f"  - Filas: {len(df_master)}")
    logger.info(f"  - Columnas: {len(df_master.columns)}")

    return df_master


def add_target_temperature(df_master: pd.DataFrame, raw_data_dir: Path) -> pd.DataFrame:
    """
    Agrega la variable target de temperatura al dataset maestro.

    Args:
        df_master: DataFrame con inputs
        raw_data_dir: Ruta al directorio con los datos raw

    Returns:
        DataFrame final con inputs y target_temperature
    """
    logger.info("\nAgregando target de temperatura...")

    # Cargar y extraer temperatura final
    df_temp = load_standardized(raw_data_dir / "eaf_temp.csv")
    df_target = get_final_temperature(df_temp)

    # Merge (inner join - solo coladas con datos completos)
    df_final = df_master.merge(df_target, on='heatid', how='inner')

    # Limpieza: eliminar columnas innecesarias
    cols_drop = ['datetime', 'positionrow', 'filter_key_date', 'measure_time']
    df_final = df_final.drop(columns=[c for c in cols_drop if c in df_final.columns])

    # Eliminar filas donde el target es nulo
    df_final = df_final.dropna(subset=['target_temperature'])

    # Rellenar nulos restantes en inputs con 0
    df_final = df_final.fillna(0)

    logger.info(f"Dataset con temperatura: {df_final.shape}")

    return df_final


def add_target_chemical(df_master: pd.DataFrame, raw_data_dir: Path) -> pd.DataFrame:
    """
    Agrega las variables target de composición química al dataset maestro.

    Args:
        df_master: DataFrame con inputs
        raw_data_dir: Ruta al directorio con los datos raw

    Returns:
        DataFrame final con inputs y targets químicos
    """
    logger.info("\nAgregando targets de composición química...")

    # Cargar y extraer composición química final
    df_chem_final = load_standardized(raw_data_dir / "eaf_final_chemical_measurements.csv")
    df_targets = get_final_chemical_composition(df_chem_final)

    if df_targets.empty:
        raise ValueError("No se pudieron extraer los targets químicos")

    # Merge (inner join - solo coladas con datos completos)
    df_final = df_master.merge(df_targets, on='heatid', how='inner')

    # Limpieza: eliminar columnas innecesarias
    cols_drop = ['datetime', 'positionrow', 'filter_key_date', 'measure_time']
    df_final = df_final.drop(columns=[c for c in cols_drop if c in df_final.columns])

    # Eliminar filas donde TODOS los targets son nulos
    target_cols = [c for c in df_final.columns if c.startswith('target_')]
    df_final = df_final.dropna(subset=target_cols, how='all')

    # <<<<<<<<<<<<<<<< CAMBIO: Aplicar Filtro de Outliers a Nivel de Dataset Maestro >>>>>>>>>>>>>>>>>>
    print("\n[PREPROCESAMIENTO QUÍMICO] Aplicando filtro de outliers (1% inferior/superior) a targets:")
    rows_initial = len(df_final)

    # Iterar sobre todos los targets y eliminar la fila si el valor es un outlier en CUALQUIER target
    for target in target_cols:
        df_final, n_removed = _filter_outliers_by_target(
            df_final,
            target,
            lower_q=0.01,
            upper_q=0.99
        )
        if n_removed > 0:
            print(f"  - Eliminadas {n_removed} filas por outlier en {target} ({len(df_final)} restantes)")

    total_removed = rows_initial - len(df_final)
    if total_removed > 0:
        print(f"Total de filas eliminadas por outliers en cualquier target: {total_removed}")

    # Rellenar nulos restantes con 0
    df_final = df_final.fillna(0)

    logger.info(f"Dataset con química: {df_final.shape}")
    logger.info(f"Targets: {target_cols}")

    return df_final


print("✅ Funciones de construcción definidas:")
print("  - build_master_dataset(): Crea dataset de inputs")
print("  - add_target_temperature(): Agrega target de temperatura")
print("  - add_target_chemical(): Agrega targets químicos (con filtrado de outliers)")

---

## PARTE 3: Feature Engineering - Dataset Secuencial

Esta sección genera el **dataset secuencial** para predicción paso a paso de temperatura.

**Pipeline de transformación:**
1. **Carga de datos**: Mediciones temporales de temperatura (eaf_temp.csv)
2. **Limpieza robusta**: Filtrado de temperaturas físicamente válidas (1000-1850°C) y eliminación de outliers por cuantiles
3. **Ordenamiento temporal**: Por HEATID y DATETIME
4. **Features dinámicas**: Temperatura actual, oxidación, posición en secuencia
5. **Target secuencial**: Temperatura del siguiente registro (shift)
6. **Fusión con variables estáticas**: Merge con dataset maestro (energía, materiales, etc.)
7. **Exportación**: `dataset_sequential_temp.csv`

**Nota sobre funciones auxiliares:** Las funciones `load_standardized()`, `build_master_dataset()` y otras funciones de agregación definidas anteriormente se reutilizan aquí para obtener las variables estáticas de cada colada.


### 3.1 Carga y Limpieza de Datos de Temperatura

In [ ]:
# =============================================================================
# PASO 1: CARGA Y LIMPIEZA DE DATOS DE TEMPERATURA
# =============================================================================

# Cargar el archivo de temperaturas con todas las mediciones
df_temp_seq = load_standardized(DATA_RAW / "eaf_temp.csv")

print(f"Archivo eaf_temp.csv cargado")
print(f"Shape original: {df_temp_seq.shape}")
print(f"\nColumnas disponibles:")
print(df_temp_seq.columns.tolist())
print(f"\nPrimeras filas:")
df_temp_seq.head(10)

In [ ]:
# =============================================================================
# DETECCIÓN AUTOMÁTICA DE COLUMNAS Y LIMPIEZA DE TIPOS
# =============================================================================

# Detectar columna de temperatura (excluir 'time' en el nombre)
cols_temp = [c for c in df_temp_seq.columns if 'temp' in c.lower() and 'time' not in c.lower()]
col_temp = cols_temp[0] if cols_temp else 'temp'
print(f"Columna de temperatura detectada: {col_temp}")

# Detectar columna de tiempo/fecha
cols_time = [c for c in df_temp_seq.columns if 'time' in c.lower() or 'date' in c.lower()]
col_datetime = cols_time[0] if cols_time else 'datetime'
print(f"Columna de tiempo detectada: {col_datetime}")

# Detectar columna de oxidación si existe
cols_ox = [c for c in df_temp_seq.columns if 'ox' in c.lower() or 'o2' in c.lower()]
col_oxidation = cols_ox[0] if cols_ox else None
print(f"Columna de oxidación detectada: {col_oxidation}")

# Convertir tipos de datos
# Temperatura a numérico
df_temp_seq[col_temp] = pd.to_numeric(df_temp_seq[col_temp], errors='coerce')

# Datetime - manejar formato con coma en milisegundos
df_temp_seq[col_datetime] = pd.to_datetime(
    df_temp_seq[col_datetime].astype(str).str.replace(',', '.', regex=False),
    errors='coerce'
)

# Oxidación a numérico si existe
if col_oxidation:
    df_temp_seq[col_oxidation] = pd.to_numeric(df_temp_seq[col_oxidation], errors='coerce')

# Mostrar tipos resultantes
print(f"\nTipos de datos después de conversión:")
print(df_temp_seq.dtypes)

# Estadísticas básicas
print(f"\nEstadísticas de temperatura:")
print(df_temp_seq[col_temp].describe())

#### 3.1.1 Limpieza Robusta de Datos de Temperatura

**CRÍTICO**: Antes de generar features y targets, aplicamos filtros para eliminar ruido del sensor:

1. **Filtro físico**: Solo temperaturas entre 1000°C y 1850°C (rango operativo del EAF)
2. **Filtro estadístico**: Eliminación de cuantiles extremos (0.5% inferior y superior)

Este paso previene que outliers del sensor contaminen el modelo.

In [ ]:
# =============================================================================
# PASO 2B: LIMPIEZA ROBUSTA DE DATOS DE TEMPERATURA (CRÍTICO)
# =============================================================================
# Aplicar filtros ANTES de generar lags/shifts para evitar contaminación

filas_inicial = len(df_temp_seq)
print(f"Filas iniciales: {filas_inicial}")

# -----------------------------------------------------------------------------
# FILTRO 1: Rango físico de temperaturas válidas para EAF
# El horno de arco eléctrico opera típicamente entre 1000°C y 1850°C
# Valores fuera de este rango son errores del sensor
# -----------------------------------------------------------------------------
TEMP_MIN_FISICA = 1000  # °C - Por debajo de esto, el acero no está fundido
TEMP_MAX_FISICA = 1850  # °C - Por encima de esto, daño al equipo

mask_fisica = (df_temp_seq[col_temp] >= TEMP_MIN_FISICA) & (df_temp_seq[col_temp] <= TEMP_MAX_FISICA)
df_temp_seq = df_temp_seq[mask_fisica].copy()

filas_despues_fisica = len(df_temp_seq)
eliminadas_fisica = filas_inicial - filas_despues_fisica
print(f"Filtro físico ({TEMP_MIN_FISICA}-{TEMP_MAX_FISICA}°C): {eliminadas_fisica} filas eliminadas")
print(f"  -> Filas restantes: {filas_despues_fisica}")

# -----------------------------------------------------------------------------
# FILTRO 2: Cuantiles para eliminar ruido extremo del sensor
# Eliminamos el 0.5% inferior y superior de las temperaturas restantes
# -----------------------------------------------------------------------------
QUANTILE_LOWER = 0.005
QUANTILE_UPPER = 0.995

q_low = df_temp_seq[col_temp].quantile(QUANTILE_LOWER)
q_high = df_temp_seq[col_temp].quantile(QUANTILE_UPPER)

mask_quantile = (df_temp_seq[col_temp] >= q_low) & (df_temp_seq[col_temp] <= q_high)
df_temp_seq = df_temp_seq[mask_quantile].copy()

filas_despues_quantile = len(df_temp_seq)
eliminadas_quantile = filas_despues_fisica - filas_despues_quantile
print(f"Filtro cuantil ({QUANTILE_LOWER:.1%}-{QUANTILE_UPPER:.1%}): {eliminadas_quantile} filas eliminadas")
print(f"  -> Rango aceptado: [{q_low:.1f}, {q_high:.1f}] °C")
print(f"  -> Filas restantes: {filas_despues_quantile}")

# Resumen de limpieza
total_eliminadas = filas_inicial - filas_despues_quantile
pct_eliminado = 100 * total_eliminadas / filas_inicial
print(f"\n{'='*60}")
print(f"RESUMEN DE LIMPIEZA DE DATOS")
print(f"{'='*60}")
print(f"Filas iniciales:      {filas_inicial:,}")
print(f"Filas eliminadas:     {total_eliminadas:,} ({pct_eliminado:.2f}%)")
print(f"Filas finales:        {filas_despues_quantile:,}")
print(f"{'='*60}")

# Estadísticas de temperatura después de limpieza
print(f"\nEstadísticas de temperatura DESPUÉS de limpieza:")
print(df_temp_seq[col_temp].describe())

### 3.2 Ordenamiento Estricto por HEATID y DATETIME

In [ ]:
# =============================================================================
# PASO 2: ORDENAMIENTO ESTRICTO POR HEATID Y DATETIME
# =============================================================================
# CRÍTICO: Sin ordenamiento correcto, la secuencia temporal no tiene sentido

# Eliminar filas con valores nulos en columnas críticas
df_temp_seq = df_temp_seq.dropna(subset=['heatid', col_datetime, col_temp])
print(f"Filas después de eliminar nulos críticos: {len(df_temp_seq)}")

# Ordenar OBLIGATORIAMENTE por heatid (colada) y luego por datetime (tiempo)
df_temp_seq = df_temp_seq.sort_values(['heatid', col_datetime]).reset_index(drop=True)

print(f"\nDataset ordenado por heatid y {col_datetime}")
print(f"Shape: {df_temp_seq.shape}")

# Verificar ordenamiento mostrando una colada de ejemplo
ejemplo_heatid = df_temp_seq['heatid'].iloc[0]
print(f"\nEjemplo de colada ordenada (heatid={ejemplo_heatid}):")
df_temp_seq[df_temp_seq['heatid'] == ejemplo_heatid][['heatid', col_datetime, col_temp]].head(10)

### 3.3 Creación de Features Dinámicas ($X_t$)

In [ ]:
# =============================================================================
# PASO 3: CREACIÓN DE FEATURES DINÁMICAS (X_t)
# =============================================================================
# Estas son las features que representan el estado actual en cada momento t

# Feature 1: Temperatura actual (X_t)
df_temp_seq['temp_actual'] = df_temp_seq[col_temp]

# Feature 2: Oxidación actual (si existe, sino rellenar con 0)
if col_oxidation:
    df_temp_seq['oxidacion_actual'] = df_temp_seq[col_oxidation].fillna(0)
else:
    df_temp_seq['oxidacion_actual'] = 0
    print("Nota: No se encontró columna de oxidación, se rellena con 0")

# Feature 3: Número de medición dentro de la colada (posición secuencial)
df_temp_seq['num_medicion'] = df_temp_seq.groupby('heatid').cumcount() + 1

# Feature 4: Tiempo transcurrido desde inicio de la colada (en minutos)
df_temp_seq['tiempo_desde_inicio'] = df_temp_seq.groupby('heatid')[col_datetime].transform(
    lambda x: (x - x.min()).dt.total_seconds() / 60
)

print("Features dinámicas creadas:")
print("  - temp_actual: Temperatura en el momento actual")
print("  - oxidacion_actual: Nivel de oxidación actual")
print("  - num_medicion: Número de medición dentro de la colada")
print("  - tiempo_desde_inicio: Minutos desde inicio de la colada")

# Mostrar ejemplo
print(f"\nEjemplo de features para colada {ejemplo_heatid}:")
df_temp_seq[df_temp_seq['heatid'] == ejemplo_heatid][[
    'heatid', col_datetime, 'temp_actual', 'oxidacion_actual', 'num_medicion', 'tiempo_desde_inicio'
]].head(10)

### 3.4 Generación del Target ($Y_t$) - Temperatura Siguiente

In [ ]:
# =============================================================================
# PASO 4: GENERACIÓN DEL TARGET (Y_t) - TEMPERATURA SIGUIENTE
# =============================================================================
# El target es la temperatura del SIGUIENTE registro dentro de la misma colada
# Usamos shift(-1) agrupado por heatid

# Target: Temperatura del siguiente momento temporal (shift negativo de 1)
df_temp_seq['target_temp_next'] = df_temp_seq.groupby('heatid')[col_temp].shift(-1)

# También guardamos el datetime del siguiente registro (para calcular horizonte)
df_temp_seq['datetime_next'] = df_temp_seq.groupby('heatid')[col_datetime].shift(-1)

print("Target generado: target_temp_next")
print("  - Representa la temperatura del siguiente registro de la misma colada")
print("  - El último registro de cada colada tendrá NaN (será eliminado)")

# Verificar: mostrar ejemplo con target
print(f"\nEjemplo de target para colada {ejemplo_heatid}:")
df_temp_seq[df_temp_seq['heatid'] == ejemplo_heatid][[
    'heatid', col_datetime, 'temp_actual', 'target_temp_next', 'datetime_next'
]].head(10)

### 3.5 Cálculo del Horizonte Temporal

In [ ]:
# =============================================================================
# PASO 5: CÁLCULO DEL HORIZONTE TEMPORAL
# =============================================================================
# Calculamos cuántos minutos faltan para la siguiente medición
# Esto es útil para que el modelo sepa a qué horizonte está prediciendo

# Horizonte: diferencia en minutos entre datetime_next y datetime actual
df_temp_seq['horizonte_minutos'] = (
    df_temp_seq['datetime_next'] - df_temp_seq[col_datetime]
).dt.total_seconds() / 60

print("Horizonte temporal calculado: horizonte_minutos")
print("  - Minutos hasta la siguiente medición")

# Estadísticas del horizonte
print(f"\nEstadísticas del horizonte temporal:")
print(df_temp_seq['horizonte_minutos'].describe())

# Mostrar ejemplo completo
print(f"\nEjemplo completo para colada {ejemplo_heatid}:")
df_temp_seq[df_temp_seq['heatid'] == ejemplo_heatid][[
    'heatid', 'num_medicion', 'temp_actual', 'target_temp_next', 'horizonte_minutos'
]].head(10)

### 3.6 Limpieza de Bordes (Eliminar Últimos Registros)

In [ ]:
# =============================================================================
# PASO 6: LIMPIEZA DE BORDES - ELIMINAR ÚLTIMOS REGISTROS DE CADA COLADA
# =============================================================================
# El último registro de cada colada no tiene target (NaN por el shift)
# Debemos eliminarlo sistemáticamente

filas_antes = len(df_temp_seq)

# Contar cuántas coladas tenemos
n_coladas = df_temp_seq['heatid'].nunique()
print(f"Número de coladas: {n_coladas}")
print(f"Filas antes de limpieza: {filas_antes}")

# Eliminar filas donde target_temp_next es NaN (últimos registros de cada colada)
df_temp_seq = df_temp_seq.dropna(subset=['target_temp_next'])

filas_despues = len(df_temp_seq)
filas_eliminadas = filas_antes - filas_despues

print(f"Filas eliminadas (últimos de cada colada): {filas_eliminadas}")
print(f"Filas después de limpieza: {filas_despues}")

# Verificar que se eliminaron aproximadamente n_coladas filas
print(f"\nVerificación: Se esperaban ~{n_coladas} eliminaciones, se eliminaron {filas_eliminadas}")

# Eliminar también filas con horizonte negativo o muy largo (outliers)
print(f"\nFiltrando horizontes anómalos...")
mask_horizonte_valido = (df_temp_seq['horizonte_minutos'] > 0) & (df_temp_seq['horizonte_minutos'] < 120)
df_temp_seq = df_temp_seq[mask_horizonte_valido]
print(f"Filas después de filtrar horizontes: {len(df_temp_seq)}")

### 3.7 Fusión con Dataset Maestro (Merge One-to-Many)

In [ ]:
# =============================================================================
# PASO 7: FUSIÓN CON DATASET MAESTRO (ONE-TO-MANY)
# =============================================================================
# Fusionamos el dataset secuencial con el dataset maestro de variables estáticas
# Las variables estáticas se REPETIRÁN en cada paso temporal (diseño esperado)

# Primero, construir el dataset maestro si no existe
print("Construyendo dataset maestro de variables estáticas...")
df_master = build_master_dataset(DATA_RAW)

print(f"\nDataset maestro shape: {df_master.shape}")
print(f"Dataset secuencial shape: {df_temp_seq.shape}")

# Fusionar: dataset secuencial es la BASE (izquierda)
# Esto replica las variables estáticas para cada medición temporal
df_sequential = df_temp_seq.merge(
    df_master,
    on='heatid',
    how='left'  # Mantener todas las filas del dataset secuencial
)

print(f"\nDataset después de fusión: {df_sequential.shape}")
print(f"  - Cada paso temporal ahora tiene acceso a las variables estáticas de la colada")

# Verificar que las variables estáticas se replicaron
print(f"\nEjemplo de replicación para colada {ejemplo_heatid}:")
cols_ejemplo = ['heatid', 'num_medicion', 'temp_actual', 'target_temp_next', 'total_energy', 'total_o2_lance']
cols_disponibles = [c for c in cols_ejemplo if c in df_sequential.columns]
df_sequential[df_sequential['heatid'] == ejemplo_heatid][cols_disponibles].head(5)

### 3.8 Preparación Final del Dataset Secuencial

In [ ]:
# =============================================================================
# PASO 8: PREPARACIÓN FINAL DEL DATASET SECUENCIAL
# =============================================================================

# Rellenar nulos con 0 (valores técnicos faltantes)
df_sequential = df_sequential.fillna(0)

# Eliminar columnas auxiliares que ya no necesitamos
cols_drop = ['datetime_next', col_datetime, 'datetime', 'positionrow', 'filter_key_date', 'measure_time']
cols_drop = [c for c in cols_drop if c in df_sequential.columns]
df_sequential = df_sequential.drop(columns=cols_drop)

# Eliminar también la columna de temperatura original (ya tenemos temp_actual)
if col_temp in df_sequential.columns and col_temp != 'temp_actual':
    df_sequential = df_sequential.drop(columns=[col_temp])

# Reorganizar columnas: heatid primero, luego features, luego target
target_col = 'target_temp_next'
cols_order = ['heatid'] + [c for c in df_sequential.columns if c not in ['heatid', target_col]] + [target_col]
df_sequential = df_sequential[cols_order]

print("="*60)
print("DATASET SECUENCIAL FINAL PARA ENTRENAMIENTO")
print("="*60)
print(f"Shape: {df_sequential.shape}")
print(f"\nColumnas ({len(df_sequential.columns)}):")
print(df_sequential.columns.tolist())

print(f"\nEstadísticas del target (temperatura siguiente):")
print(df_sequential['target_temp_next'].describe())

print(f"\nPrimeras filas:")
df_sequential.head()

In [ ]:
# =============================================================================
# GUARDAR DATASET SECUENCIAL
# =============================================================================

output_path = DATA_PROCESSED / "dataset_sequential_temp.csv"
df_sequential.to_csv(output_path, index=False)

print(f"Dataset secuencial guardado en: {output_path}")
print(f"  - Filas: {len(df_sequential)}")
print(f"  - Columnas: {len(df_sequential.columns)}")
print(f"  - Coladas únicas: {df_sequential['heatid'].nunique()}")
print(f"  - Mediciones promedio por colada: {len(df_sequential) / df_sequential['heatid'].nunique():.1f}")

### 3.9 Resumen del Dataset Secuencial

**Dataset generado:** `dataset_sequential_temp.csv`

**Estructura:**
- **Una fila por medición temporal** (múltiples filas por colada)
- **Features dinámicas**: `temp_actual`, `oxidacion_actual`, `num_medicion`, `tiempo_desde_inicio`, `horizonte_minutos`
- **Features estáticas**: Variables del dataset maestro (energía, materiales, etc.) replicadas en cada paso
- **Target**: `target_temp_next` - temperatura de la siguiente medición

**Uso para entrenamiento:**
```python
# Cargar dataset
df = pd.read_csv(DATA_PROCESSED / "dataset_sequential_temp.csv")

# Separar features y target
X = df.drop(columns=['heatid', 'target_temp_next'])
y = df['target_temp_next']

# IMPORTANTE: Para split train/test, agrupar por heatid para evitar data leakage
```

In [ ]:
# =============================================================================
# VERIFICACIÓN FINAL Y ANÁLISIS EXPLORATORIO
# =============================================================================

print("="*60)
print("ANÁLISIS DEL DATASET SECUENCIAL")
print("="*60)

# Distribución de mediciones por colada
mediciones_por_colada = df_sequential.groupby('heatid').size()
print(f"\nMediciones por colada:")
print(mediciones_por_colada.describe())

# Correlación entre features dinámicas y target
features_dinamicas = ['temp_actual', 'oxidacion_actual', 'num_medicion', 'tiempo_desde_inicio', 'horizonte_minutos']
features_disponibles = [f for f in features_dinamicas if f in df_sequential.columns]

print(f"\nCorrelación de features dinámicas con target:")
for f in features_disponibles:
    corr = df_sequential[f].corr(df_sequential['target_temp_next'])
    print(f"  {f}: {corr:.4f}")

# Visualización
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Distribución del target
axes[0, 0].hist(df_sequential['target_temp_next'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Temperatura siguiente (°C)')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución del Target')

# 2. Temp actual vs Temp siguiente
axes[0, 1].scatter(df_sequential['temp_actual'], df_sequential['target_temp_next'], alpha=0.1, s=1)
axes[0, 1].plot([1500, 1750], [1500, 1750], 'r--', label='y=x')
axes[0, 1].set_xlabel('Temperatura actual (°C)')
axes[0, 1].set_ylabel('Temperatura siguiente (°C)')
axes[0, 1].set_title('Temperatura Actual vs Siguiente')
axes[0, 1].legend()

# 3. Distribución del horizonte
axes[1, 0].hist(df_sequential['horizonte_minutos'], bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Horizonte (minutos)')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribución del Horizonte Temporal')

# 4. Mediciones por colada
axes[1, 1].hist(mediciones_por_colada, bins=30, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Número de mediciones')
axes[1, 1].set_ylabel('Frecuencia (coladas)')
axes[1, 1].set_title('Mediciones por Colada')

plt.tight_layout()
plt.show()

print("\nDataset secuencial listo para entrenamiento de modelos de series temporales.")

---

## PARTE 3.5: Feature Engineering - Dataset Químico

En esta sección generamos el dataset para predecir la **composición química final** del acero.

**Objetivo:** Crear `dataset_final_chemical.csv` con:
- **Inputs:** Variables del proceso (materiales, energía, gases, etc.)
- **Targets:** Composición química final (C, Mn, Si, P, S, Cu, Cr, Mo, Ni)

**Pipeline:**
1. Cargar mediciones químicas finales (`eaf_final_chemical_measurements.csv`)
2. Extraer y limpiar los targets químicos
3. Reutilizar/generar el dataset maestro de inputs
4. Fusionar inputs con targets químicos
5. Limpieza final y guardado

### 3.5.1 Carga de Mediciones Químicas Finales

Cargamos el archivo con las mediciones de composición química realizadas **al final del proceso EAF**.

In [ ]:
# =============================================================================
# PASO 1: CARGAR MEDICIONES QUÍMICAS FINALES
# =============================================================================

df_chem_final = load_standardized(DATA_RAW / "eaf_final_chemical_measurements.csv")

print(f"Archivo cargado: eaf_final_chemical_measurements.csv")
print(f"Shape: {df_chem_final.shape}")
print(f"\nColumnas disponibles:")
print(df_chem_final.columns.tolist())
print(f"\nPrimeras filas:")
df_chem_final.head()

### 3.5.2 Extracción de Targets Químicos

Extraemos las columnas de composición química que servirán como variables target:
- **valc**: Carbono
- **valmn**: Manganeso
- **valsi**: Silicio
- **valp**: Fósforo
- **vals**: Azufre
- **valcu**: Cobre
- **valcr**: Cromo
- **valmo**: Molibdeno
- **valni**: Níquel

In [ ]:
# =============================================================================
# PASO 2: EXTRACCIÓN Y LIMPIEZA DE TARGETS QUÍMICOS
# =============================================================================

# Elementos químicos a extraer como targets
chemical_elements = [
    'valc', 'valmn', 'valsi', 'valp', 'vals',
    'valcu', 'valcr', 'valmo', 'valni'
]

# Verificar qué columnas existen en el archivo
available_elements = [col for col in chemical_elements if col in df_chem_final.columns]
missing_elements = [col for col in chemical_elements if col not in df_chem_final.columns]

print(f"Elementos disponibles: {available_elements}")
if missing_elements:
    print(f"Elementos NO encontrados: {missing_elements}")

# Seleccionar heatid y elementos químicos disponibles
cols_to_select = ['heatid'] + available_elements
df_targets = df_chem_final[cols_to_select].copy()

# Convertir a numérico (manejo de valores mal formateados)
for col in available_elements:
    df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce')

# Renombrar con prefijo target_
rename_dict = {col: f'target_{col}' for col in available_elements}
df_targets = df_targets.rename(columns=rename_dict)

# Eliminar duplicados por heatid (conservar el último registro)
df_targets = df_targets.drop_duplicates(subset=['heatid'], keep='last')

print(f"\nTargets extraídos: {len(df_targets)} coladas")
print(f"Columnas target: {[c for c in df_targets.columns if c.startswith('target_')]}")
print(f"\nEstadísticas de los targets:")
df_targets.describe()

### 3.5.3 Dataset Maestro de Inputs

Reutilizamos la función `build_master_dataset()` para generar el dataset de inputs.

**Nota:** Si `df_master` ya existe en memoria (de la PARTE 3), lo reutilizamos para evitar recálculos.

In [ ]:
# =============================================================================
# PASO 3: GENERAR/REUTILIZAR DATASET MAESTRO DE INPUTS
# =============================================================================

# Verificar si df_master ya existe en memoria
if 'df_master' not in dir() or df_master is None:
    print("Generando dataset maestro desde cero...")
    df_master = build_master_dataset(DATA_RAW)
else:
    print("Reutilizando df_master existente en memoria.")

print(f"\nDataset maestro (inputs): {df_master.shape}")
print(f"Columnas: {df_master.columns.tolist()[:10]}... (y {len(df_master.columns)-10} más)")

### 3.5.4 Fusión de Inputs con Targets Químicos

Realizamos un **inner join** entre el dataset maestro y los targets químicos.

Solo conservamos las coladas que tienen **tanto** datos de proceso **como** mediciones químicas finales.

In [ ]:
# =============================================================================
# PASO 4: FUSIÓN DE INPUTS CON TARGETS QUÍMICOS
# =============================================================================

# Merge (inner join - solo coladas con datos completos)
df_chemical = df_master.merge(df_targets, on='heatid', how='inner')

print(f"Dataset maestro: {len(df_master)} coladas")
print(f"Targets químicos: {len(df_targets)} coladas")
print(f"Dataset fusionado: {len(df_chemical)} coladas")
print(f"\nPérdida por merge: {len(df_master) - len(df_chemical)} coladas sin mediciones químicas")

### 3.5.5 Limpieza Final y Guardado

Aplicamos la limpieza final:
1. Eliminar columnas innecesarias (datetime, positionrow, etc.)
2. Eliminar filas donde **todos** los targets son nulos
3. Rellenar nulos en inputs con 0
4. Guardar el dataset en `data/processed/dataset_final_chemical.csv`

In [ ]:
# =============================================================================
# PASO 5: LIMPIEZA FINAL Y GUARDADO
# =============================================================================

# Columnas a eliminar (metadatos innecesarios para el modelo)
cols_drop = ['datetime', 'positionrow', 'filter_key_date', 'measure_time']
df_chemical = df_chemical.drop(columns=[c for c in cols_drop if c in df_chemical.columns])
print(f"Columnas eliminadas: {[c for c in cols_drop if c in df_master.columns]}")

# Identificar columnas target
target_cols = [c for c in df_chemical.columns if c.startswith('target_')]
print(f"Columnas target: {target_cols}")

# Eliminar filas donde TODOS los targets son nulos
rows_before = len(df_chemical)
df_chemical = df_chemical.dropna(subset=target_cols, how='all')
rows_dropped = rows_before - len(df_chemical)
print(f"\nFilas eliminadas (todos targets nulos): {rows_dropped}")

# Rellenar nulos en inputs con 0
null_count_before = df_chemical.isnull().sum().sum()
df_chemical = df_chemical.fillna(0)
print(f"Valores nulos rellenados con 0: {null_count_before}")

# Guardar el dataset
output_path_chemical = DATA_PROCESSED / "dataset_final_chemical.csv"
df_chemical.to_csv(output_path_chemical, index=False)

print(f"\n" + "="*60)
print(f"DATASET QUÍMICO GUARDADO EXITOSAMENTE")
print(f"="*60)
print(f"Ruta: {output_path_chemical}")
print(f"Shape final: {df_chemical.shape}")

### 3.5.6 Análisis Exploratorio del Dataset Químico

Verificación rápida del dataset generado.

In [ ]:
# =============================================================================
# ANÁLISIS EXPLORATORIO DEL DATASET QUÍMICO
# =============================================================================

print("="*60)
print("RESUMEN DEL DATASET QUÍMICO")
print("="*60)

# Información general
print(f"\n📊 DIMENSIONES")
print(f"  - Filas (coladas): {len(df_chemical):,}")
print(f"  - Columnas totales: {len(df_chemical.columns)}")
print(f"  - Features de input: {len(df_chemical.columns) - len(target_cols)}")
print(f"  - Variables target: {len(target_cols)}")

# Estadísticas de los targets
print(f"\n🎯 ESTADÍSTICAS DE TARGETS QUÍMICOS")
print(df_chemical[target_cols].describe().round(4))

# Verificar nulos
print(f"\n🔍 VERIFICACIÓN DE NULOS")
null_counts = df_chemical.isnull().sum()
if null_counts.sum() == 0:
    print("  ✅ No hay valores nulos en el dataset")
else:
    print(f"  ⚠️ Columnas con nulos: {null_counts[null_counts > 0].to_dict()}")

# Distribución de targets (valores extremos)
print(f"\n📈 RANGOS DE TARGETS")
for col in target_cols:
    min_val = df_chemical[col].min()
    max_val = df_chemical[col].max()
    mean_val = df_chemical[col].mean()
    print(f"  {col}: min={min_val:.4f}, max={max_val:.4f}, mean={mean_val:.4f}")

print(f"\n✅ Dataset químico listo para modelado en PARTE 5")

---

## PARTE 4: Modelado de Temperatura (Dataset Secuencial)

En esta sección entrenamos modelos para predecir la **temperatura del siguiente paso temporal** usando el dataset secuencial generado en la PARTE 3.

**Modelos disponibles:**
- **XGBoost Regressor**: Gradient boosting optimizado
- **Random Forest Regressor**: Ensemble de árboles de decisión
- **Linear Regression**: Modelo base lineal

**Pipeline de entrenamiento:**
1. Carga del dataset secuencial (`dataset_sequential_temp.csv`)
2. Preparación de features dinámicas y estáticas
3. **Split Train/Test por HEATID** (GroupShuffleSplit para evitar data leakage)
4. Entrenamiento del modelo seleccionado
5. Evaluación con métricas (RMSE, R², MAE)
6. Visualización de resultados

**IMPORTANTE - Validación por Grupos:**
Para evitar *data leakage*, usamos `GroupShuffleSplit` que garantiza que todas las mediciones de una misma colada (heatid) estén completamente en train o en test, nunca mezcladas.

---

## PARTE 5: Modelado de Composición Química

La predicción química es más delicada debido a:
- **Outliers extremos**: Valores atípicos que distorsionan las métricas
- **Data Leakage potencial**: Usar el valor inicial del mismo elemento como feature
- **Múltiples targets**: 9 elementos químicos diferentes

**Estrategias implementadas:**
1. **Exclusión inteligente de features**: Se excluye el valor inicial del mismo elemento
2. **Capping de outliers**: Se eliminan el 1% inferior y superior (cuantiles 0.01-0.99)
3. **Imputación de NaNs**: Features con valor 0, filas con target NaN eliminadas

### 5.0 Configuración de Variables para Modelado Químico

Definimos las variables específicas para el modelado de composición química:
- **CHEMICAL_TARGETS**: Targets químicos a predecir
- **CHEMICAL_SPECS**: Rangos de especificación para control de calidad

In [ ]:
# =============================================================================
# CONFIGURACIÓN PARA MODELADO QUÍMICO
# =============================================================================

# -----------------------------------------------------------------------------
# TARGETS QUÍMICOS
# Valores FINALES de composición química a predecir
# -----------------------------------------------------------------------------
CHEMICAL_TARGETS = [
    'target_valc',              # Carbono final
    'target_valmn',             # Manganeso final
    'target_valsi',             # Silicio final
    'target_valp',              # Fósforo final
    'target_vals',              # Azufre final
    'target_valcu',             # Cobre final
    'target_valcr',             # Cromo final
    'target_valmo',             # Molibdeno final
    'target_valni'              # Níquel final
]

# -----------------------------------------------------------------------------
# ESPECIFICACIONES QUÍMICAS
# Rangos de especificación (min, max) para valores FINALES - Control de calidad
# -----------------------------------------------------------------------------
CHEMICAL_SPECS = {
    'target_valc':  (0.05, 0.50),    # Carbono: 0.05% - 0.50%
    'target_valmn': (0.30, 1.50),    # Manganeso: 0.30% - 1.50%
    'target_valsi': (0.10, 0.60),    # Silicio: 0.10% - 0.60%
    'target_valp':  (0.001, 0.025),  # Fósforo: 0.001% - 0.025%
    'target_vals':  (0.001, 0.025),  # Azufre: 0.001% - 0.025%
    'target_valcu': (0.001, 0.030),  # Cobre: 0.001% - 0.030%
    'target_valcr': (0.001, 0.030),  # Cromo: 0.001% - 0.030%
    'target_valmo': (0.001, 0.010),  # Molibdeno: 0.001% - 0.010%
    'target_valni': (0.001, 0.030)   # Níquel: 0.001% - 0.030%
}

print("✅ Configuración química cargada:")
print(f"  - CHEMICAL_TARGETS: {len(CHEMICAL_TARGETS)} elementos")
print(f"  - CHEMICAL_SPECS: {len(CHEMICAL_SPECS)} especificaciones")

### 5.1 Funciones Auxiliares para Modelos Químicos

Funciones específicas para el manejo de datos químicos.

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES PARA MODELOS QUÍMICOS
# =============================================================================

def load_chemical_data() -> pd.DataFrame:
    """
    Carga el dataset específico para modelos químicos.
    
    Returns:
        DataFrame con los datos químicos procesados
    """
    # El archivo 'dataset_final_chemical.csv' ahora ya fue filtrado de outliers
    return load_and_clean_data("dataset_final_chemical.csv")


def get_chemical_features(df: pd.DataFrame, target: str) -> List[str]:
    """
    Obtiene la lista de features disponibles para entrenamiento químico,
    aplicando exclusiones inteligentes para evitar data leakage.

    EXCLUSIONES:
    1. El propio target (target_valc, etc.)
    2. El valor inicial del mismo elemento (valc, etc.) - EVITA DATA LEAKAGE
    3. Todos los demás targets químicos
    4. Columna de identificador (heatid)

    Args:
        df: DataFrame con los datos
        target: Target químico a predecir (ej: 'target_valc')

    Returns:
        Lista de features válidas para entrenamiento

    Example:
        >>> features = get_chemical_features(df, 'target_valc')
        >>> # Excluirá 'valc' y 'target_valc' de las features
    """
    # Determinar el feature inicial correspondiente al target
    # target_valc -> valc, target_valmn -> valmn, etc.
    initial_feature_to_exclude = target.replace('target_', '')

    # Lista de columnas a excluir
    exclude_cols = ['heatid', target]

    # Excluir TODOS los targets químicos (no solo el actual)
    exclude_cols += [col for col in df.columns if col.startswith('target_')]

    # Excluir el feature inicial del mismo elemento
    if initial_feature_to_exclude in df.columns:
        exclude_cols.append(initial_feature_to_exclude)

    # Filtrar: usar INPUT_FEATURES que estén en df y NO estén en exclusiones
    available = [
        col for col in INPUT_FEATURES
        if col in df.columns and col not in exclude_cols
    ]

    return available


# La función cap_outliers ha sido ELIMINADA de esta celda,
# ya que su lógica se movió a la función add_target_chemical en la PARTE 3.5.

print("✅ Funciones auxiliares químicas definidas:")
print("  - load_chemical_data(): Carga dataset_final_chemical.csv")
print("  - get_chemical_features(): Selección inteligente de features")

### 5.2 Función Principal: train_chemical_model()

Entrena un modelo para predecir un elemento químico específico con manejo robusto de outliers.

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL: ENTRENAMIENTO DE MODELO QUÍMICO
# =============================================================================

def train_chemical_model(
    target: str,
    model_type: str = 'xgboost',
    n_estimators: int = None,
    max_depth: int = None,
    learning_rate: float = None,
    test_size: float = None,
    random_state: int = None,
    save_model: bool = True,
    feature_list: List[str] = None,
    outlier_quantiles: Tuple[float, float] = (0.01, 0.99)
) -> Tuple[Any, Dict[str, float], List[str], pd.DataFrame, pd.Series, np.ndarray, Optional[Path]]:
    """
    Entrena un modelo de predicción de composición química FINAL.
    
    IMPORTANTE: Implementa limpieza de outliers para evitar R² negativos.
    
    Args:
        target: Elemento químico a predecir ('target_valc', 'target_valmn', etc.)
        model_type: Tipo de modelo ('xgboost', 'random_forest', 'linear')
        n_estimators: Número de estimadores para tree models
        max_depth: Profundidad máxima de árboles
        learning_rate: Learning rate para XGBoost
        test_size: Proporción de datos para test
        random_state: Semilla para reproducibilidad
        save_model: Si True, guarda el modelo en disco
        feature_list: Lista personalizada de features (si None, usa selección inteligente)
        outlier_quantiles: Tuple (lower, upper) para capping de outliers (Aplica en la fase de FE, aquí se omite)

    Returns:
        Tuple con:
        - model: Modelo entrenado
        - metrics: Dict con RMSE, R², MAE
        - feature_names: Lista de features usadas
        - X_test: DataFrame de features de test
        - y_test: Series con valores reales de test
        - y_pred: Array con predicciones
        - model_path: Path al modelo guardado
    """
    # -------------------------------------------------------------------------
    # VALIDACIÓN DEL TARGET
    # -------------------------------------------------------------------------
    if target not in CHEMICAL_TARGETS:
        raise ValueError(
            f"Target '{target}' no válido.\n"
            f"Debe ser uno de: {CHEMICAL_TARGETS}"
        )

    # Usar hiperparámetros por defecto si no se especifican
    n_estimators = n_estimators or DEFAULT_HYPERPARAMS['n_estimators']
    max_depth = max_depth or DEFAULT_HYPERPARAMS['max_depth']
    learning_rate = learning_rate or DEFAULT_HYPERPARAMS['learning_rate']
    test_size = test_size or DEFAULT_HYPERPARAMS['test_size']
    random_state = random_state or DEFAULT_HYPERPARAMS['random_state']

    # Obtener nombre limpio del elemento
    element_name = target.replace('target_', '').upper()

    print(f"\n{'='*60}")
    print(f"ENTRENAMIENTO DE MODELO QUÍMICO - {element_name}")
    print(f"{'='*60}")
    print(f"Target: {target}")
    print(f"Modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")

    # Mostrar especificación si existe
    if target in CHEMICAL_SPECS:
        min_spec, max_spec = CHEMICAL_SPECS[target]
        print(f"Especificación: [{min_spec:.3f}, {max_spec:.3f}]")

    # -------------------------------------------------------------------------
    # PASO 1: Cargar datos
    # -------------------------------------------------------------------------
    logger.info("Cargando datos químicos...")
    df = load_chemical_data()

    # Verificar que el target existe
    if target not in df.columns:
        raise KeyError(
            f"La columna target '{target}' no existe en el dataset.\n"
            f"Columnas disponibles: {[c for c in df.columns if 'target' in c]}"
        )

    print(f"\nDataset cargado: {df.shape}")

    # -------------------------------------------------------------------------
    # PASO 2: Selección inteligente de features
    # -------------------------------------------------------------------------
    if feature_list is None:
        feature_cols = get_chemical_features(df, target)
    else:
        # Aplicar exclusiones a lista personalizada
        initial_feature = target.replace('target_', '')
        feature_cols = [
            f for f in feature_list
            if f in df.columns and f != target and f != initial_feature
        ]

    X = df[feature_cols].copy()
    y = df[target].copy()

    print(f"Features seleccionadas: {len(feature_cols)}")
    print(f"  (Excluido '{target.replace('target_', '')}' para evitar data leakage)")

    # -------------------------------------------------------------------------
    # PASO 3: Eliminar filas donde el target es NaN
    # -------------------------------------------------------------------------
    rows_initial = len(X)
    mask_not_null = y.notnull()
    X = X[mask_not_null]
    y = y[mask_not_null]
    rows_after_null = len(X)

    if rows_after_null < rows_initial:
        print(f"Eliminadas {rows_initial - rows_after_null} filas con target NaN")

    # -------------------------------------------------------------------------
    # PASO 4: CAPPING DE OUTLIERS (CRÍTICO)
    # ELIMINADO: La lógica de filtrado de outliers se movió a la función add_target_chemical
    # en la PARTE 3. El dataset cargado ya debe estar limpio.
    # -------------------------------------------------------------------------
    
    # -------------------------------------------------------------------------
    # PASO 5: Imputación de NaNs en features
    # -------------------------------------------------------------------------
    X = X.fillna(0)
    
    # Limpiar valores infinitos
    X = X.replace([np.inf, -np.inf], 0)
    y = y.replace([np.inf, -np.inf], np.nan).dropna()
    X = X.loc[y.index]  # Sincronizar índices
    
    if len(X) == 0:
        raise ValueError(f"El dataset para '{target}' quedó vacío después de la limpieza.")
    
    print(f"\nDataset final: {len(X)} muestras, {len(feature_cols)} features")
    print(f"Estadísticas del target: min={y.min():.4f}, max={y.max():.4f}, mean={y.mean():.4f}")
    
    # -------------------------------------------------------------------------
    # PASO 6: Split Train/Test
    # -------------------------------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    print(f"\nSplit Train/Test:")
    print(f"  - Train: {len(X_train)} muestras")
    print(f"  - Test: {len(X_test)} muestras")
    
    # -------------------------------------------------------------------------
    # PASO 7: Crear y entrenar modelo
    # -------------------------------------------------------------------------
    logger.info(f"Entrenando modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")
    
    if model_type == 'linear':
        model = LinearRegression()
    elif model_type == 'random_forest':
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state,
            n_jobs=-1
        )
    elif model_type == 'xgboost':
        model = XGBRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            random_state=random_state,
            n_jobs=-1
        )
    else:
        raise ValueError(f"Modelo no reconocido: {model_type}")
    
    print(f"\nEntrenando {MODEL_DISPLAY_NAMES.get(model_type, model_type)}...")
    model.fit(X_train, y_train)
    print("✅ Entrenamiento completado")
    
    # -------------------------------------------------------------------------
    # PASO 8: Predecir y evaluar
    # -------------------------------------------------------------------------
    y_pred = model.predict(X_test)
    metrics = calculate_metrics(y_test, y_pred)
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS DE EVALUACIÓN - {element_name}")
    print(f"{'='*60}")
    print(f"  RMSE: {metrics['RMSE']:.6f}")
    print(f"  R²:   {metrics['R2']:.4f}")
    print(f"  MAE:  {metrics['MAE']:.6f}")
    
    # Verificación de calidad de R²
    if metrics['R2'] < 0:
        print(f"  ⚠️  ADVERTENCIA: R² negativo. Modelo peor que la media.")
    elif metrics['R2'] < 0.3:
        print(f"  ⚠️  R² bajo. Considera ajustar hiperparámetros.")
    
    print(f"{'='*60}")
    
    # -------------------------------------------------------------------------
    # PASO 9: Guardar modelo
    # -------------------------------------------------------------------------
    model_path = None
    if save_model:
        import pickle
        from datetime import datetime
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        element_short = target.replace('target_', '')
        model_name = f"chem_{element_short}_{model_type}_{timestamp}"
        
        # Crear subdirectorio
        model_subdir = MODELS_DIR / model_name
        model_subdir.mkdir(exist_ok=True)
        
        model_path = model_subdir / "model.joblib"
        metadata_path = model_subdir / "metadata.json"
        
        # Guardar modelo
        joblib.dump(model, model_path)
        
        # Guardar metadatos
        metadata = {
            "model_type": model_type,
            "model_display_name": MODEL_DISPLAY_NAMES.get(model_type, model_type),
            "features": feature_cols,
            "hyperparameters": {
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "learning_rate": learning_rate,
                "test_size": test_size,
                "random_state": random_state
            },
            "metrics": metrics,
            "timestamp": timestamp,
            "target": target,
            "element": element_short,
            "n_samples_train": len(X_train),
            "n_samples_test": len(X_test),
            "outlier_quantiles": list(outlier_quantiles)
        }
        
        if target in CHEMICAL_SPECS:
            metadata["specification"] = {
                "min": CHEMICAL_SPECS[target][0],
                "max": CHEMICAL_SPECS[target][1]
            }
        
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)
        
        # Guardar también en chemical_results para compatibilidad con dashboard
        chemical_results_dir = CHEMICAL_RESULTS_DIR
        chemical_results_dir.mkdir(parents=True, exist_ok=True)
        
        importance_df = get_feature_importance(model, feature_cols, model_type)
        if importance_df is not None:
            results_data = {
                'y_test': y_test,
                'y_pred': y_pred,
                'importance_df': importance_df,
                'metrics': metrics
            }
            results_file = chemical_results_dir / f"results_{element_short}.pkl"
            with open(results_file, 'wb') as f:
                pickle.dump(results_data, f)
        
        print(f"\n💾 Modelo guardado en: {model_subdir}")
    
    return model, metrics, feature_cols, X_test, y_test, y_pred, model_path


print("✅ Función principal definida: train_chemical_model()")

### 5.3 Visualización Específica para Química

Funciones de visualización que incluyen las especificaciones químicas de calidad.

In [ ]:
# =============================================================================
# VISUALIZACIÓN ESPECÍFICA PARA QUÍMICA
# =============================================================================

def plot_chemical_predictions(
    y_test: np.ndarray,
    y_pred: np.ndarray,
    metrics: Dict[str, float],
    target: str
) -> plt.Figure:
    """
    Genera scatter plot con especificaciones químicas marcadas.
    
    Args:
        y_test: Valores reales
        y_pred: Valores predichos
        metrics: Diccionario con métricas
        target: Nombre del target (ej: 'target_valc')
    
    Returns:
        Figure de matplotlib
    """
    element_name = target.replace('target_', '').upper()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.5, color='steelblue', edgecolors='white', linewidth=0.5)
    
    # Línea de predicción perfecta
    min_val = min(np.min(y_test), np.min(y_pred))
    max_val = max(np.max(y_test), np.max(y_pred))
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Predicción Perfecta')
    
    # Marcar especificaciones si existen
    if target in CHEMICAL_SPECS:
        min_spec, max_spec = CHEMICAL_SPECS[target]
        ax.axhline(y=min_spec, color='green', linestyle=':', alpha=0.7, label=f'Spec Min: {min_spec}')
        ax.axhline(y=max_spec, color='green', linestyle=':', alpha=0.7, label=f'Spec Max: {max_spec}')
        ax.axvline(x=min_spec, color='green', linestyle=':', alpha=0.7)
        ax.axvline(x=max_spec, color='green', linestyle=':', alpha=0.7)
        
        # Zona de especificación
        ax.fill_between([min_spec, max_spec], min_spec, max_spec, 
                        color='green', alpha=0.1, label='Zona Óptima')
    
    ax.set_xlabel(f'Valor Real - {element_name} (%)', fontsize=12)
    ax.set_ylabel(f'Valor Predicho - {element_name} (%)', fontsize=12)
    ax.set_title(f'Predicción vs Real - {element_name}', fontsize=14, fontweight='bold')
    
    # Métricas en recuadro
    textstr = f"RMSE: {metrics['RMSE']:.6f}\nR²: {metrics['R2']:.4f}\nMAE: {metrics['MAE']:.6f}"
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    
    ax.legend(loc='lower right', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_chemical_summary(results_dict: Dict[str, Dict]) -> plt.Figure:
    """
    Genera un gráfico resumen de todos los modelos químicos entrenados.
    
    Args:
        results_dict: Diccionario con resultados por target
                     {target: {'metrics': {...}, 'model': ...}, ...}
    
    Returns:
        Figure de matplotlib
    """
    targets = list(results_dict.keys())
    r2_values = [results_dict[t]['metrics']['R2'] for t in targets]
    rmse_values = [results_dict[t]['metrics']['RMSE'] for t in targets]
    
    # Limpiar nombres
    labels = [t.replace('target_', '').upper() for t in targets]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Subplot 1: R² por elemento
    ax1 = axes[0]
    colors = ['green' if r2 > 0.5 else 'orange' if r2 > 0 else 'red' for r2 in r2_values]
    bars1 = ax1.bar(labels, r2_values, color=colors, edgecolor='white')
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax1.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Umbral bueno (0.5)')
    ax1.set_ylabel('R²', fontsize=12)
    ax1.set_title('Coeficiente de Determinación (R²) por Elemento', fontsize=12)
    ax1.set_ylim(-0.5, 1.0)
    ax1.legend()
    
    # Añadir valores en barras
    for bar, val in zip(bars1, r2_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{val:.3f}', ha='center', fontsize=9)
    
    # Subplot 2: RMSE por elemento
    ax2 = axes[1]
    bars2 = ax2.bar(labels, rmse_values, color='steelblue', edgecolor='white')
    ax2.set_ylabel('RMSE', fontsize=12)
    ax2.set_title('Error Cuadrático Medio (RMSE) por Elemento', fontsize=12)
    
    # Añadir valores en barras
    for bar, val in zip(bars2, rmse_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{val:.4f}', ha='center', fontsize=9, rotation=45)
    
    plt.suptitle('Resumen de Modelos Químicos', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig


print("✅ Funciones de visualización química definidas:")
print("  - plot_chemical_predictions(): Scatter con especificaciones")
print("  - plot_chemical_summary(): Resumen de todos los modelos")

### 5.4 Ejecución: Entrenamiento de Modelos Químicos

Entrenamos modelos para los principales elementos químicos.

In [ ]:
# =============================================================================
# ENTRENAMIENTO DE MODELOS QUÍMICOS
# =============================================================================

# Seleccionar elementos a entrenar (principales 5 elementos)
# Puedes cambiar esta lista para incluir más o menos elementos
TARGETS_TO_TRAIN = [
    'target_valc',    # Carbono
    'target_valmn',   # Manganeso
    'target_valsi',   # Silicio
    'target_valp',    # Fósforo
    'target_vals',    # Azufre
]

# Diccionario para almacenar resultados
chemical_results = {}

print(f"{'='*60}")
print("ENTRENAMIENTO DE MODELOS QUÍMICOS")
print(f"{'='*60}")
print(f"Elementos a entrenar: {len(TARGETS_TO_TRAIN)}")
print(f"Targets: {[t.replace('target_', '').upper() for t in TARGETS_TO_TRAIN]}")
print(f"{'='*60}\n")

# Entrenar modelo para cada elemento
for target in TARGETS_TO_TRAIN:
    try:
        model, metrics, features, X_test, y_test, y_pred, path = train_chemical_model(
            target=target,
            model_type='xgboost',
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            save_model=True,
            outlier_quantiles=(0.01, 0.99)  # Eliminar 1% extremos
        )
        
        # Guardar resultados
        chemical_results[target] = {
            'model': model,
            'metrics': metrics,
            'features': features,
            'X_test': X_test,
            'y_test': y_test,
            'y_pred': y_pred,
            'path': path
        }
        
    except Exception as e:
        print(f"\n❌ Error entrenando {target}: {e}\n")
        continue

print(f"\n{'='*60}")
print(f"RESUMEN: {len(chemical_results)}/{len(TARGETS_TO_TRAIN)} modelos entrenados exitosamente")
print(f"{'='*60}")

In [ ]:
# =============================================================================
# VISUALIZACIÓN DE RESULTADOS QUÍMICOS
# =============================================================================

# Mostrar gráfico resumen de todos los modelos
if chemical_results:
    fig_summary = plot_chemical_summary(chemical_results)
    plt.show()
    
    # Tabla resumen de métricas
    print(f"\n{'='*70}")
    print("TABLA RESUMEN DE MÉTRICAS POR ELEMENTO")
    print(f"{'='*70}")
    print(f"{'Elemento':<12} {'RMSE':>12} {'R²':>12} {'MAE':>12} {'Spec Min':>10} {'Spec Max':>10}")
    print("-" * 70)
    
    for target, result in chemical_results.items():
        element = target.replace('target_', '').upper()
        metrics = result['metrics']
        
        if target in CHEMICAL_SPECS:
            min_spec, max_spec = CHEMICAL_SPECS[target]
            print(f"{element:<12} {metrics['RMSE']:>12.6f} {metrics['R2']:>12.4f} {metrics['MAE']:>12.6f} {min_spec:>10.3f} {max_spec:>10.3f}")
        else:
            print(f"{element:<12} {metrics['RMSE']:>12.6f} {metrics['R2']:>12.4f} {metrics['MAE']:>12.6f} {'N/A':>10} {'N/A':>10}")
    
    print(f"{'='*70}")
else:
    print("⚠️ No hay resultados para mostrar")

In [ ]:
# =============================================================================
# GRÁFICOS INDIVIDUALES POR ELEMENTO
# =============================================================================

# Mostrar gráfico de predicción para cada elemento entrenado
for target, result in chemical_results.items():
    fig = plot_chemical_predictions(
        result['y_test'],
        result['y_pred'],
        result['metrics'],
        target
    )
    plt.show()
    
    # Mostrar feature importance para este modelo
    fig_imp = plot_feature_importance(
        result['model'],
        result['features'],
        model_type='xgboost',
        top_n=10,
        title=f"Importancia de Variables - {target.replace('target_', '').upper()}"
    )
    if fig_imp:
        plt.show()
    
    print("-" * 60)